# SupplyChain class

In [1]:
import numpy as np
import pandas as pd

from climada.util.api_client import Client
from climada_petals.engine import SupplyChain
from climada.entity import ImpfTropCyclone, ImpactFuncSet
from climada.engine.impact_calc import ImpactCalc

client = Client()

This tutorial shows how to use the `SupplyChain` class of CLIMADA. This class allows assessing indirect impacts via Input-Output (IO) modeling. Before diving into this class, it is highly recommended that the user first understands how direct impact is calculated with CLIMADA. This requires getting familiar with the `Exposures`, `Hazard` and `Impact` classes.

This tutorial shows how to set up a global supply chain risk analysis due to tropical cyclones hitting the United States.

# 1. Calculate direct economic impacts

The first step is to conduct a direct impact analysis. To do so, we need to define an exposure, an hazard and a vulnerability. In this tutorial we will load the LitPop exposure for the USA from CLIMADA Data API.

In [2]:
exp_usa = client.get_litpop('USA')

2023-09-27 15:06:38,917 - climada.entity.exposures.base - INFO - Reading /Users/aciullo/climada/data/exposures/litpop/LitPop_150arcsec_USA/v2/LitPop_150arcsec_USA.hdf5


/Users/aciullo/opt/anaconda3/envs/climada_env/lib/python3.9/pickle.py:1717: UserWarning: Unpickling a shapely <2.0 geometry object. Please save the pickle again; shapely 2.1 will not have this compatibility.
  setstate(state)


Then, we load a probabilistic hazard set of tropical cyclones for the USA from the CLIMADA Data API.

In [3]:
tc_usa = client.get_hazard('tropical_cyclone', properties={'country_iso3alpha':'USA', 'climate_scenario':'historical'})

2023-09-27 15:06:49,402 - climada.hazard.base - INFO - Reading /Users/aciullo/climada/data/hazard/tropical_cyclone/tropical_cyclone_10synth_tracks_150arcsec_USA_1980_2020/v2/tropical_cyclone_10synth_tracks_150arcsec_USA_1980_2020.hdf5


Then we define vulnerability by loading impact functions for tropical cyclone in the USA:

In [4]:
# Define impact function
impf_tc = ImpfTropCyclone.from_emanuel_usa()
impf_set = ImpactFuncSet()
impf_set.append(impf_tc)
impf_set.check()

And we finally calculate impacts.

In [5]:
# Calculate direct impacts to the USA due to TC
imp_calc = ImpactCalc(exp_usa, impf_set, tc_usa)
direct_impact_usa = imp_calc.impact()

2023-09-27 15:06:49,555 - climada.entity.exposures.base - INFO - No specific impact function column found for hazard TC. Using the anonymous 'impf_' column.
2023-09-27 15:06:49,555 - climada.entity.exposures.base - INFO - Matching 643099 exposures with 503715 centroids.
2023-09-27 15:06:49,562 - climada.util.coordinates - INFO - No exact centroid match found. Reprojecting coordinates to nearest neighbor closer than the threshold = 100
2023-09-27 15:06:50,515 - climada.util.coordinates - WARNING - Distance to closest centroid is greater than 100km for 129140 coordinates.
2023-09-27 15:06:50,542 - climada.engine.impact_calc - INFO - Calculating impact for 1327884 assets (>0) and 43560 events.


# 2. Calculate indirect economic impacts

## 2.1 Instantiate a `SupplyChain` object by loading the Multi-Regional Input-Output Table of interest.

`SupplyChain` computes indirect economic impacts via Input-Output (IO) modeling. At the core of IO modeling lies an Input-Output Table. `SupplyChain` uses the [pymrio](https://pymrio.readthedocs.io/en/latest/) python package to download, parse and save Multi-Regional Input Output Tables (MRIOTs). In principle, any IO table can be loaded and used, as long as the structure is consistent with those internally supported by `SupplyChain`, which are:
- [EXIOBASE3](https://zenodo.org/record/3583071#.Y_4zZuzMK3I) (1995-2011; 44 countries; 163 industries)​
- [WIOD16](https://www.rug.nl/ggdc/valuechain/wiod/) (2000-2014; 43 countries; 56 industries)​
- [OECD21](https://www.oecd.org/sti/ind/inter-country-input-output-tables.htm) (1995-2018; 66 countries; 45 industries)​

These MRIOTs can be downloaded, parsed and saved automatically.

The first step is to instantiate a `SupplyChain` class. This can be done by passing a customized MRIOT or by calling the `from_mriot` class method to use one of the supported MRIOTs.

In [6]:
supchain = SupplyChain.from_mriot(mriot_type='WIOD16', mriot_year=2011)

2023-09-27 15:06:51,329 - climada.util.files_handler - INFO - Downloading https://dataverse.nl/api/access/datafile/199104 to file /Users/aciullo/climada/data/MRIOT/WIOD16/downloads/199104


898kKB [00:47, 19.0kKB/s]                           


The instatiated class now has an `mriot` attribute, which is a pymrio `IOSystem` object. As such, one can access several info of the MRIOT incl. regions, sectors, total production, transaction matrix and final demand. Please see the pymrio project on how to make best use of all the provided functions. 

For example, one can access regions, sectors and IOT data:

In [7]:
# regions
supchain.mriot.get_regions()

Index(['AUS', 'AUT', 'BEL', 'BGR', 'BRA', 'CAN', 'CHE', 'CHN', 'CYP', 'CZE',
       'DEU', 'DNK', 'ESP', 'EST', 'FIN', 'FRA', 'GBR', 'GRC', 'HRV', 'HUN',
       'IDN', 'IND', 'IRL', 'ITA', 'JPN', 'KOR', 'LTU', 'LUX', 'LVA', 'MEX',
       'MLT', 'NLD', 'NOR', 'POL', 'PRT', 'ROU', 'RUS', 'SVK', 'SVN', 'SWE',
       'TUR', 'TWN', 'USA', 'ROW'],
      dtype='object', name='region')

In [8]:
# sectors
supchain.mriot.get_sectors()

Index(['Crop and animal production, hunting and related service activities',
       'Forestry and logging', 'Fishing and aquaculture',
       'Mining and quarrying',
       'Manufacture of food products, beverages and tobacco products',
       'Manufacture of textiles, wearing apparel and leather products',
       'Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials',
       'Manufacture of paper and paper products',
       'Printing and reproduction of recorded media',
       'Manufacture of coke and refined petroleum products ',
       'Manufacture of chemicals and chemical products ',
       'Manufacture of basic pharmaceutical products and pharmaceutical preparations',
       'Manufacture of rubber and plastic products',
       'Manufacture of other non-metallic mineral products',
       'Manufacture of basic metals',
       'Manufacture of fabricated metal products, except machinery and equipment',
       

In [9]:
# transaction matrix
supchain.mriot.Z

region                                                                                                                   AUS  \
sector                                                    Crop and animal production, hunting and related service activities   
region sector                                                                                                                  
AUS    Crop and animal production, hunting and related...                                       10954.209508                   
       Forestry and logging                                                                       113.905445                   
       Fishing and aquaculture                                                                     22.267974                   
       Mining and quarrying                                                                       232.437685                   
       Manufacture of food products, beverages and tob...                                        1662.225516                   
...                                                                                                      ...                   
ROW    Education                                                                                    1.305193                   
       Human health and social work activities                                                      0.535337                   
       Other service activities                                                                     1.856064                   
       Activities of households as employers; undiffer...                                           0.027816                   
       Activities of extraterritorial organizations an...                                           0.000000                   

region                                                                          \
sector                                                    Forestry and logging   
region sector                                                                    
AUS    Crop and animal production, hunting and related...           337.318807   
       Forestry and logging                                         127.867760   
       Fishing and aquaculture                                        0.025312   
       Mining and quarrying                                           2.323883   
       Manufacture of food products, beverages and tob...             4.046841   
...                                                                        ...   
ROW    Education                                                      0.022124   
       Human health and social work activities                        0.038688   
       Other service activities                                       0.058495   
       Activities of households as employers; undiffer...             0.000064   
       Activities of extraterritorial organizations an...             0.000000   

region                                                                             \
sector                                                    Fishing and aquaculture   
region sector                                                                       
AUS    Crop and animal production, hunting and related...              166.404066   
       Forestry and logging                                              0.588653   
       Fishing and aquaculture                                          21.871529   
       Mining and quarrying                                              8.353050   
       Manufacture of food products, beverages and tob...               78.987732   
...                                                                           ...   
ROW    Education                                                         0.093113   
       Human health and social work activities                           0.141195   
       Other service activities                                          0.149588   
       Activities of households as emp

In [10]:
# final demand
supchain.mriot.Y

final demand
region sector                                                           
AUS    Crop and animal production, hunting and related...   14467.050267
       Forestry and logging                                   342.352502
       Fishing and aquaculture                               1787.071582
       Mining and quarrying                                 19445.035584
       Manufacture of food products, beverages and tob...   51480.063474
...                                                                  ...
ROW    Education                                           483744.763934
       Human health and social work activities             382325.468496
       Other service activities                            234071.579585
       Activities of households as employers; undiffer...   14600.614007
       Activities of extraterritorial organizations an...      33.142482

[2464 rows x 1 columns]

In [11]:
# total production
supchain.mriot.x

total production
region sector                                                              
AUS    Crop and animal production, hunting and related...      69566.630223
       Forestry and logging                                     2719.293635
       Fishing and aquaculture                                  3113.703184
       Mining and quarrying                                   250207.439237
       Manufacture of food products, beverages and tob...      92033.592425
...                                                                     ...
ROW    Education                                              508147.713351
       Human health and social work activities                399362.733830
       Other service activities                               367396.468789
       Activities of households as employers; undiffer...      22003.013824
       Activities of extraterritorial organizations an...         33.142482

[2464 rows x 1 columns]

## 2.2 Assign stock exposure and impact to MRIOT countries-sectors

After loading the MRIOT, one needs to translate the direct impacts previously calculated - which are defined at an arbitrary spatial resolution - into impacts to sectors and countries defined by the MRIOT. To do this one needs to know what countries and sectors the used exposure represents. 

The first is straightforward, as exposure contains latitude and longitude information, and even a regional id that often defines the country of interest.

The latter needs input from the user, who needs to know/assess what sectors in the MRIOT the used exposure represents. For example, assuming the `LitPop` exposure is representative of the service sector, and assuming that sub-sectors at positions 26 to 56 in `WIOD16` do represent this sector, then one can translate spatially disaggregated impacts into country/sector impacts as follows:

In [12]:
impacted_secs = supchain.mriot.get_sectors()[range(26,56)].tolist()
supchain.calc_shock_to_sectors(exp_usa, direct_impact_usa, impacted_secs)

Which creates the attributes `self.secs_exp`, `self.secs_imp`, and `self.secs_shock`. The first two show `Exposure` and `Impact` values at the country-sector level. This translation is accomplished assuming that exposure/impact of an affected sector is proportional to this sector's contribution to the overall production of all affected sectors. For example, if the total (spatially distributed) exposed value is 100, and there are two affected sectors, A (whose production is 2) and B (whose production is 8), then sector A has an exposure of 20 and sector B has an exposure of 80. The same reasoning is applied to the distributions of direct impacts.

One can easily check that `self.secs_exp`, `self.secs_imp` have the same total values of `Exposure` and `Impact` and that this only involves the directly hit countries-sectors :

In [13]:
# exposure
print(
      exp_usa.gdf.value.sum(),
      supchain.secs_exp.sum().sum(),
      supchain.secs_exp.loc[:, ('USA', impacted_secs)].sum().sum(),
      )

# impact
print(supchain.secs_imp.sum().sum(),
      supchain.secs_imp.loc[:, ('USA', impacted_secs)].sum().sum(),
      direct_impact_usa.imp_mat.sum().sum())

65384554304412.63 65384554304412.64 65384554304412.64
16543899066892.074 16543899066892.07 16543899066892.068


The attribute `self.secs_shock` is proportional to the ratio between `self.secs_imp` and `self.secs_exp`, so `self.secs_shock` is a number between 0 and 1. `self.secs_shock` will be used in the indirect impact calculation to assses how much production loss is experienced by each sector.

In terms of structure, it is a dataframe with columns the MRIOT's columns, and with index the event-ids of the hazard events that have *non-zero impacts*:

In [14]:
supchain.secs_shock.loc[:, ('USA', impacted_secs)].head()

region            USA  \
sector   Construction   
event_id                
10594        0.000352   
10597        0.000634   
10600        0.000694   
10603        0.000032   
10604        0.000071   

region                                                                            \
sector   Wholesale and retail trade and repair of motor vehicles and motorcycles   
event_id                                                                           
10594                                              0.000352                        
10597                                              0.000634                        
10600                                              0.000694                        
10603                                              0.000032                        
10604                                              0.000071                        

region                                                              \
sector   Wholesale trade, except of motor vehicles and motorcycles   
event_id                                                             
10594                                              0.000352          
10597                                              0.000634          
10600                                              0.000694          
10603                                              0.000032          
10604                                              0.000071          

region                                                           \
sector   Retail trade, except of motor vehicles and motorcycles   
event_id                                                          
10594                                              0.000352       
10597                                              0.000634       
10600                                              0.000694       
10603                                              0.000032       
10604                                              0.000071       

region                                                               \
sector   Land transport and transport via pipelines Water transport   
event_id                                                              
10594                                      0.000352        0.000352   
10597                                      0.000634        0.000634   
10600                                      0.000694        0.000694   
10603                                      0.000032        0.000032   
10604                                      0.000071        0.000071   

region                                                                        \
sector   Air transport Warehousing and support activities for transportation   
event_id                                                                       
10594         0.000352                                           0.000352      
10597         0.000634                                           0.000634      
10600         0.000694                                           0.000694      
10603         0.000032                                           0.000032      
10604         0.000071                                           0.000071      

region                                  \
sector   Postal and courier activities   
event_id                                 
10594                         0.000352   
10597                         0.000634   
10600                         0.000694   
10603                         0.000032   
10604                         0.000071   

region                                              ...  \
sector   Accommodation and food service activities  ...   
event_id                                            ...   
10594                                     0.000352  ...   
10597                                     0.000634  ...   
10600                                     0.000694  ...   
10603                                     0.000032  ...   
10604                                     0.000071  ...   

region  

By default, `self.secs_shock` is exactly the ratio between `Impact` and `Exposure`, which results in the same shock across sectors for a given event:

In [15]:
# let's try the first three events
for event_id in supchain.secs_shock.index[:3]:
    imp_event = direct_impact_usa.at_event[direct_impact_usa.event_id == event_id][0]
    print(imp_event / exp_usa.gdf.value.sum(), supchain.secs_shock.loc[event_id, ('USA', impacted_secs)].values[0])

0.0003524059371639781 0.0003524059371639781
0.0006340759642834555 0.0006340759642834556
0.0006936414424383894 0.0006936414424383894


This practically means that the fraction of *production* losses is assumed to be equal to the fractions of *stock* losses, since `Impact` and `Exposure` typically refer to stocks in CLIMADA. However, since depending on the sector one can reasonably expect production losses to be proportionally higher or lower than stock losses, a `shock_factor` argument can also be passed to define - for each sector - how much should production shocks be higher/lower than stocks shocks (i.e., the mere `Impact` / `Exposure` ratio):

In [16]:
shock_factor = pd.DataFrame(np.repeat(1, supchain.mriot.x.shape[0]), 
                            index=supchain.mriot.x.index,
                            columns=['shock'])

# randomly generated for this tutorial
shock_facs_service_USA = np.array([
                                   0.38324804, 1.15930626, 0.73846477, 0.5430206 , 0.54147014,
                                   0.28362671, 0.53829353, 1.95367016, 1.33675622, 0.42285787,
                                   0.86974667, 1.4685637 , 1.24804793, 0.56915521, 0.43723048,
                                   0.23372398, 0.69268485, 0.74130451, 0.74739106, 1.18719852,
                                   1.02203697, 1.0412411 , 0.09315484, 1.23612412, 0.55947349,
                                   0.8608431, 0.58983156, 1.13137055, 0.93014364, 0.39092134
                                   ])

shock_factor.loc[('USA', impacted_secs), :] = shock_facs_service_USA

# supply shock factors when calculating sectorial shocks
supchain.calc_shock_to_sectors(exp_usa, direct_impact_usa, impacted_secs, shock_factor.values.flatten())

In [17]:
supchain.secs_shock.loc[:, ('USA', impacted_secs)].head()

region            USA  \
sector   Construction   
event_id                
10594        0.000135   
10597        0.000243   
10600        0.000266   
10603        0.000012   
10604        0.000027   

region                                                                            \
sector   Wholesale and retail trade and repair of motor vehicles and motorcycles   
event_id                                                                           
10594                                              0.000409                        
10597                                              0.000735                        
10600                                              0.000804                        
10603                                              0.000037                        
10604                                              0.000082                        

region                                                              \
sector   Wholesale trade, except of motor vehicles and motorcycles   
event_id                                                             
10594                                              0.000260          
10597                                              0.000468          
10600                                              0.000512          
10603                                              0.000024          
10604                                              0.000052          

region                                                           \
sector   Retail trade, except of motor vehicles and motorcycles   
event_id                                                          
10594                                              0.000191       
10597                                              0.000344       
10600                                              0.000377       
10603                                              0.000017       
10604                                              0.000039       

region                                                               \
sector   Land transport and transport via pipelines Water transport   
event_id                                                              
10594                                      0.000191        0.000100   
10597                                      0.000343        0.000180   
10600                                      0.000376        0.000197   
10603                                      0.000017        0.000009   
10604                                      0.000038        0.000020   

region                                                                        \
sector   Air transport Warehousing and support activities for transportation   
event_id                                                                       
10594         0.000190                                           0.000688      
10597         0.000341                                           0.001239      
10600         0.000373                                           0.001355      
10603         0.000017                                           0.000062      
10604         0.000038                                           0.000139      

region                                  \
sector   Postal and courier activities   
event_id                                 
10594                         0.000471   
10597                         0.000848   
10600                         0.000927   
10603                         0.000043   
10604                         0.000095   

region                                              ...  \
sector   Accommodation and food service activities  ...   
event_id                                            ...   
10594                                     0.000149  ...   
10597                                     0.000268  ...   
10600                                     0.000293  ...   
10603                                     0.000013  ...   
10604                                     0.000030  ...   

region  

Even though the default values (all one) for the shock factors are correct only in the (uncommon) case in which CLIMADA's direct impacts already express production losses, a proper assignment of the shock factors requires extensive expert knowledge on how a sector's production responds to the sector's stock losses. Therefore, it is recommended to change these values only when detailed knowledge about the relationship between stock and production losses is available.

## 2.3 Calculate the propagation of production losses

After sectorial production shocks are defined, one can calculate the extent of the shocks and how these propagate through the supply chain. This can be done with the via the `self.calc_impacts` method by specifying the `ghosh` or `leontief` approach. Both approached can be run on the same instantiation of `SupplyChain` and results are stored in a dictionary with keys equal to the method's name. In addition, `self.calc_impacts` calls `self.calc_shock_to_sectors` if `self.secs_exp`, `self.secs_imp` and `self.secs_shock` are not defined, in which case the respective arguments need to be passed to `self.calc_impacts`.

In [18]:
supchain.calc_impacts(io_approach='ghosh')
supchain.calc_impacts(io_approach='leontief')

This creates the `supchain_imp` dictionary:

In [19]:
supchain.supchain_imp.keys()

dict_keys(['ghosh', 'leontief'])

In [20]:
supchain.supchain_imp['ghosh'].head()

region                                                                  AUS  \
sector   Crop and animal production, hunting and related service activities   
event_id                                                                      
10594                                              0.108493                   
10597                                              0.195209                   
10600                                              0.213548                   
10603                                              0.009819                   
10604                                              0.021834                   

region                                                                      \
sector   Forestry and logging Fishing and aquaculture Mining and quarrying   
event_id                                                                     
10594                0.003844                0.005600             0.375871   
10597                0.006916                0.010075             0.676297   
10600                0.007566                0.011022             0.739829   
10603                0.000348                0.000507             0.034016   
10604                0.000774                0.001127             0.075643   

region                                                                 \
sector   Manufacture of food products, beverages and tobacco products   
event_id                                                                
10594                                              0.163791             
10597                                              0.294706             
10600                                              0.322391             
10603                                              0.014823             
10604                                              0.032963             

region                                                                  \
sector   Manufacture of textiles, wearing apparel and leather products   
event_id                                                                 
10594                                              0.016370              
10597                                              0.029454              
10600                                              0.032220              
10603                                              0.001481              
10604                                              0.003294              

region                                                                                                                                    \
sector   Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials   
event_id                                                                                                                                   
10594                                              0.017644                                                                                
10597                                              0.031746                                                                                
10600                                              0.034728                                                                                
10603                                              0.001597                                                                                
10604                                              0.003551                                                                                

region                                            \
sector   Manufacture of paper and paper products   
event_id                                           
10594                                   0.023483   
10597                                   0.042252   
10600                                   0.046221   
10603                                   0.002125   
10604                                   0.004726   

region                    

In [21]:
supchain.supchain_imp['leontief'].head()

region                                                                  AUS  \
sector   Crop and animal production, hunting and related service activities   
event_id                                                                      
10594                                              0.046333                   
10597                                              0.083367                   
10600                                              0.091198                   
10603                                              0.004193                   
10604                                              0.009324                   

region                                                                      \
sector   Forestry and logging Fishing and aquaculture Mining and quarrying   
event_id                                                                     
10594                0.004260                0.001743             0.921647   
10597                0.007665                0.003136             1.658299   
10600                0.008385                0.003431             1.814080   
10603                0.000386                0.000158             0.083408   
10604                0.000857                0.000351             0.185479   

region                                                                 \
sector   Manufacture of food products, beverages and tobacco products   
event_id                                                                
10594                                              0.053181             
10597                                              0.095688             
10600                                              0.104677             
10603                                              0.004813             
10604                                              0.010703             

region                                                                  \
sector   Manufacture of textiles, wearing apparel and leather products   
event_id                                                                 
10594                                              0.014059              
10597                                              0.025295              
10600                                              0.027671              
10603                                              0.001272              
10604                                              0.002829              

region                                                                                                                                    \
sector   Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials   
event_id                                                                                                                                   
10594                                              0.007054                                                                                
10597                                              0.012692                                                                                
10600                                              0.013884                                                                                
10603                                              0.000638                                                                                
10604                                              0.001420                                                                                

region                                            \
sector   Manufacture of paper and paper products   
event_id                                           
10594                                   0.031176   
10597                                   0.056094   
10600                                   0.061364   
10603                                   0.002821   
10604                                   0.006274   

region                    

values of the dictionary are `pandas.DataFrame` with indexes equal to the hazard events id *leading to non-zero impact* and columns representing the countries-sectors in the MRIOT. Numbers are expressed **in the same unit** as the used MRIOT and they represent global production losses due to tropical cyclones in the USA.

The unit can be checked by doing:

In [22]:
supchain.mriot.unit

unit
region sector                                                   
AUS    Crop and animal production, hunting and related...  M.EUR
       Forestry and logging                                M.EUR
       Fishing and aquaculture                             M.EUR
       Mining and quarrying                                M.EUR
       Manufacture of food products, beverages and tob...  M.EUR
...                                                          ...
ROW    Education                                           M.EUR
       Human health and social work activities             M.EUR
       Other service activities                            M.EUR
       Activities of households as employers; undiffer...  M.EUR
       Activities of extraterritorial organizations an...  M.EUR

[2464 rows x 1 columns]

As an example, the 10 largest impacted Swiss sectors from such events according to Ghosh are:

In [23]:
supchain.supchain_imp['ghosh'].loc[:,('CHE', slice(None))].max(0).sort_values(ascending=False)[:10]

region  sector                                                                                  
CHE     Manufacture of basic pharmaceutical products and pharmaceutical preparations                43.403808
        Wholesale trade, except of motor vehicles and motorcycles                                   19.367726
        Computer programming, consultancy and related activities; information service activities    10.181569
        Financial service activities, except insurance and pension funding                           8.498905
        Land transport and transport via pipelines                                                   8.182419
        Manufacture of computer, electronic and optical products                                     7.778191
        Insurance, reinsurance and pension funding, except compulsory social security                6.988161
        Construction                                                                                 6.603282
        Electricity, ga

While for Leontief are:

In [24]:
supchain.supchain_imp['leontief'].loc[:,('CHE', slice(None))].max(0).sort_values(ascending=False)[:10]

region  sector                                                                                        
CHE     Insurance, reinsurance and pension funding, except compulsory social security                     75.087467
        Manufacture of computer, electronic and optical products                                          21.647353
        Wholesale trade, except of motor vehicles and motorcycles                                         13.481617
        Manufacture of basic pharmaceutical products and pharmaceutical preparations                      12.788959
        Manufacture of chemicals and chemical products                                                    10.844197
        Legal and accounting activities; activities of head offices; management consultancy activities    10.724539
        Financial service activities, except insurance and pension funding                                 9.149769
        Electricity, gas, steam and air conditioning supply                          